# Introduction

>  A continuation is an abstract representation of the control state of a program. [...] Let’s say continuation is a data structure that represents the computational process at a given point in the process’s execution, we could save an execution state and continue the computational process latter. Seems like lambda function in Python could be used for this since we could pass a lambda function as parameters and call them later. [ref](https://coderscat.com/understanding-recursion-and-continuation-with-python/)

## Continuation-passing style

A function in continuation-passing style (CPS) has an extra argument which is the continuation, i.e., the reference for a unary function. When the original function ends, it returns by calling the continuation giving it an appropriate value. This value represents, usually, the current state of the computation.

The simplest continuation is one that simply returns the given value. Usually this is useful at the end of the computation, where the given value is the desired result. Let's call it the «end continuation»,

In [ ]:
end_cont = lambda x: x

Let's implement a function $f(x,y)$ that computes $2x+y$ in CPS-style.

For that, we create a sum and a multiplication function, that are called in sequence by `f`,

In [ ]:
def add(x, y, cont=end_cont):
  return lambda: cont(x+y)

def mul(x, y, cont=end_cont):
  return lambda: cont(x*y)

# compute 2*x + y
def f(x, y, cont=end_cont):
  cont2 = lambda new_val, y=y, c=cont: add(new_val, y, c)
  return mul(2, x, cont2)

If we try to evaluate `f` for some input,

In [ ]:
f(3, 10)

<function __main__.mul.<locals>.<lambda>()>

it returns a sequence of lambdas, as expected. In order to find the computed value, we need to activate those lambdas: 

In [ ]:
f(3, 10)()()

16

A [trampoline](https://en.wikipedia.org/wiki/Trampoline_(computing)) is a function that performs this task, i.e., it loops all lambdas feeding the result of one to the input of the next,

In [ ]:
def trampoline(f, *args):
  """ iteratively invokes thunk-returning functions """
  v = f(*args)
  while callable(v):
    v = v()
  return v

This works since every continuation call is a tail call, i.e., nothing happens at the current function after the continuation is called.

To compute the final result using a trampoline:

In [ ]:
trampoline(f, 3, 10)

16

Let's try a more complex example, computing the hypotenuse of a right triangle:

In [ ]:
def sqrt(x, cont=end_cont):
  return lambda: cont(x**0.5)

def pow2(x, cont=end_cont):
  return lambda: cont(x**2)

def pyth(x, y, cont=end_cont):
  return pow2(x, 
              lambda x2: pow2(y, 
                              lambda y2: add(x2, y2, 
                                             lambda a2b2: sqrt(a2b2)
                                            )
                             )
             )  

In [ ]:
print(pyth(6, 8)()()()())
print(trampoline(pyth, 6, 8))

10.0
10.0


Alternatively,

In [ ]:
def pyth(x, y, cont=end_cont):
  cont_sqrt = lambda a2b2  : sqrt(a2b2)
  cont_add  = lambda x2, y2: add(x2, y2, cont_sqrt)
  cont_y2   = lambda x2    : pow2(y, lambda y2: cont_add(x2,y2))
  cont_x2   =                pow2(x, lambda x2: cont_y2(x2))
  return cont_x2

print(trampoline(pyth, 6, 8))

10.0


These are examples of continuations that return to the original called, which are denoted _bounded continuations_.

> Unbounded continuations _do not return to their caller_. They express a flow of computation where results flow in one direction, without ever returning. [...] when unbounded continuations can be treated as first-class values in a language, they become so powerful that they can be used to implement pretty much any control-flow feature you can imagine - exceptions, threads, coroutines and so on. This is precisely what continuations are sometimes used for in the implementation of functional languages, where CPS-transform is one of the compilation stages. [ref](https://eli.thegreenplace.net/2017/on-recursion-continuations-and-trampolines/)

## Simulating Exceptions

Consider the following functions that

+ returns a value

+ raises an error

+ binds two operations in sequence (monadid style)

+ tries one operation, and in case of error, executes a second operation

In [ ]:
def return_(x):
  return lambda ret, err: ret(x)

def raise_(x):
  return lambda ret, err: err(x)

def bind(op1, op2):
  """ executes op1 then op2, outputs error if either fails """
  return lambda ret, err: op1(lambda res: op2(res)(ret, err), err)

def try_(op1, op2):
  """ executes op1, and if op1 fails, executes op2 """
  return lambda ret, err: op1(ret, lambda res: op2(res)(ret, err))

Let's define the return and raise operation as just print commands,

In [ ]:
ret = lambda x: print('ok: ', x)  
err = lambda x: print('error: ', x) 

run_cps = lambda op: op(ret, err)

and a function that might produce an error, depending on its parameters,

In [ ]:
def safe_div(x,y):
  def f(ret, err): 
    if y==0:
      err('div by zero')
    else:
      ret(x/y)
  return f

In the next example, we apply two safe division commands in sequence, first divide $x/2$ then divide the result by $y$,

In [ ]:
op = lambda x,y: bind(safe_div(x,2), 
                      lambda a: safe_div(a,y))
run_cps(op(30,2))  
run_cps(op(30,0))

ok:  7.5
error:  div by zero


An example of exception: try to divide $x/y$, and if an error occurs, divide $x/10$,

In [ ]:
op = lambda x,y: try_(safe_div(x,y), 
                      lambda _: safe_div(x,10))

run_cps(op(30,0))

ok:  3.0


In the next example, in case of a problem, it would raise an exception:

In [ ]:
op = lambda x,y: try_(safe_div(x,y), 
                      lambda msg: raise_(msg))

run_cps(op(30,2))
run_cps(op(30,0))

ok:  15.0
error:  div by zero


## Tail-Call Optimization

Tail-recursive functions are functions where the recursive call is the last command to be executed. These functions can be optimized to prevent the increasing size of the call stack, since the last call can be replaced by the next recursive call. These process is called tail-call optimization (TCO) which is done automatically by some programming languages.

Let's introduce a helper function that returns the current depth of the call stack. This will be useful to check how the call stack is being used by our recursive functions.

In [ ]:
import traceback

def factory_current_depth():
  """ eval the current call stack depth """
  current_depth = len(traceback.extract_stack())
  return lambda: len(traceback.extract_stack())-current_depth

depth = factory_current_depth()
show_depth = lambda *state: print(f"{depth()*'*'} : {state}")

Let's test it using a standard tail-recursive implementation of the factorial function,

In [ ]:
def fact_tail(n, fac=1):
  show_depth(n)
  if n==0:
    return fac
  return fact_tail(n-1, n*fac)

print(fact_tail(8))

** : (8,)
*** : (7,)
**** : (6,)
***** : (5,)
****** : (4,)
******* : (3,)
******** : (2,)
********* : (1,)
********** : (0,)
40320


We can observe how the call stack is increasing for each recursive call. Python, by [design](https://stackoverflow.com/questions/13591970), does not do tail-call optimization, unlike most functional programming languages.

However, we can implement the factorial in a continuation-passing style,

In [ ]:
def fact_cps(n, cont=end_cont):
  show_depth(n) 
  if n == 0:
    return cont(1) 
  return lambda: fact_cps(n-1, lambda fac: cont(n*fac))

print(trampoline(fact_cps, 8))

*** : (8,)
**** : (7,)
**** : (6,)
**** : (5,)
**** : (4,)
**** : (3,)
**** : (2,)
**** : (1,)
**** : (0,)
40320


And, as we just saw, the call stack is not growing. We have achieved TCO using `fact_cps` along with the trampoline function!

In this case, the implementation can be simplified to a sequence of thunk-returning functions, removing the continuation context,

In [ ]:
def fact_tco(n, fac=1):
  show_depth(n) 
  if n == 0:
    return fac 
  return lambda: fact_tco(n-1, n*fac)

print(trampoline(fact_tco, 8))

*** : (8,)
**** : (7,)
**** : (6,)
**** : (5,)
**** : (4,)
**** : (3,)
**** : (2,)
**** : (1,)
**** : (0,)
40320


This simplification is even greater for the Fibonacci sequence, that uses two recursive parameters.

Let's check the traditional tail-recursive definition, the cps solution and the simplified implementation:

In [ ]:
def fib_tail(n, a=1, b=1):
  show_depth(n) 
  if n < 2:
    return b
  else:
    return fib_tail(n - 1, b, a+b)

print(fib_tail(8))

** : (8,)
*** : (7,)
**** : (6,)
***** : (5,)
****** : (4,)
******* : (3,)
******** : (2,)
********* : (1,)
34


In [ ]:
def fib_cps(n, cont=end_cont):
  """ binary recursion, CPS version """
  show_depth(n) 
  if n < 2:
    return cont(1)
  return lambda: fib_cps(n-1, lambda x:
                                 lambda: fib_cps(n-2, lambda y:
                                                        lambda: cont(x+y)))

def fib_cps(n, a=1, b=1, cont=end_cont):
  """ linear recursion, CPS version """
  show_depth(n,a,b) 
  if n < 2:
    return cont(b) 
  return lambda: fib_cps(n-1, b, a+b, lambda x: cont(x))

print(trampoline(fib_cps, 8))

*** : (8, 1, 1)
**** : (7, 1, 2)
**** : (6, 2, 3)
**** : (5, 3, 5)
**** : (4, 5, 8)
**** : (3, 8, 13)
**** : (2, 13, 21)
**** : (1, 21, 34)
34


In [ ]:
def fib_tco(n, a=1, b=1):
  show_depth(n, a, b) 
  if n < 2:
    return b
  else:
    return lambda: fib_tco(n-1, b, a+b)

print(trampoline(fib_tco, 8))   

*** : (8, 1, 1)
**** : (7, 1, 2)
**** : (6, 2, 3)
**** : (5, 3, 5)
**** : (4, 5, 8)
**** : (3, 8, 13)
**** : (2, 13, 21)
**** : (1, 21, 34)
34


The next example computes mergesort with continuations. 

In this case, since the number of recursion calls are proportional to $\log n$, the call stack growth is not problematic.

In [ ]:
def merge(xs, ys):
  result = []
  while xs or ys:
    if not xs or not ys:
       return result + xs + ys
    x, *xtail = xs
    y, *ytail = ys
    if x < y:
      result.append(x)
      xs = xtail
    else:
      result.append(y)
      ys = ytail
  return result

def mergesort_cps(xs, cont=end_cont):
  if len(xs) <= 1:
    return cont(xs)
  mid = int(len(xs) / 2)
  return mergesort_cps(xs[:mid],
                       lambda L: mergesort_cps(xs[mid:], 
                                               lambda R: cont(merge(L, R))))

In [ ]:
trampoline(mergesort_cps, [7,5,3,8,1,2,0,10,3])

[0, 1, 2, 3, 3, 5, 7, 8, 10]

## Implementing Backtracking


The idea is:

+ if a search does ont produce a result, do not call the continuation

+ if a search produces several results, call the continuation for each result

+ the continuation must flow!

This first example executes a continuation for each element of the given list:

In [ ]:
def member(xs, cont):
  if xs:
    x, *xs = xs
    cont(x)
    return member(xs, cont)

Let's choose a print as the continuation:

In [ ]:
cont_show = lambda *x: print(*x, end='; ')  
member([1,2,3], cont_show)

1; 2; 3; 

Searches can be combined:

In [ ]:
def search2(xs, ys):
  member(xs, 
         lambda x: member(ys, 
                          lambda y: cont_show(x,y)))
  
search2([1,2,3], 'abcd')

1 a; 1 b; 1 c; 1 d; 2 a; 2 b; 2 c; 2 d; 3 a; 3 b; 3 c; 3 d; 

And we can apply filters that cut the search if some predicate is not satisfied:

In [ ]:
cont_nothing = lambda x: None # do nothing continuation

def filt(p, cont):
  return lambda *x: cont(*x) if p(x[0]) else cont_nothing(x)

def search3(xs, ys):
  member(xs, 
         lambda x: member(ys, 
                          lambda y: filt(lambda x: x%2==0, cont_show)(x,y)))

search3([1,2,3], 'abcd')

2 a; 2 b; 2 c; 2 d; 

The next function produces all possible insertions of an element into a list:

In [ ]:
def ins(xs, z, cont):
  if not xs:
    cont([z])
  else:
    cont([z]+xs)
    x, *xs = xs
    ins(xs, z, lambda ys: cont([x]+ys))

ins([1,2,3], 4, cont_show)  

[4, 1, 2, 3]; [1, 4, 2, 3]; [1, 2, 4, 3]; [1, 2, 3, 4]; 

The previous function can be used to compute permutations:

In [ ]:
def perm(xs, cont):
  if not xs:
    cont([])
  else:
    x, *xs = xs
    perm(xs, lambda ys: ins(ys, x, cont))

perm([1,2,3], cont_show)    

[1, 2, 3]; [2, 1, 3]; [2, 3, 1]; [1, 3, 2]; [3, 1, 2]; [3, 2, 1]; 

If we'd like to have carry state across the search, instead of just making some side-effect, we need to have a continuation that receives a value and a state, and outputs a new state:

In [ ]:
def member_st(xs, cont, state):
  if not xs:
    return state
  else:
    x, *xs = xs
    new_state = cont(x, state)
    return member_st(xs, cont, new_state)

In [ ]:
# eg, map elements of list
cont_mul10 = lambda value, state: state+[10*value]
member_st([1,2,3], cont_mul10, [])

[10, 20, 30]

Let's define a continuation that simply adds the current value to the previous state:

In [ ]:
cont_keep = lambda info, state: state+[info]

And let's refactor `ins` and `perm` to return all searches, instead of printing them:

In [ ]:
def ins(xs, z, cont, state):
  if not xs:
    return cont([z], state)
  else:
    new_state = cont([z]+xs, state)
    x, *xs = xs
    return ins(xs, z, lambda ys, st2: cont([x]+ys, st2), new_state)

ins([1,2,3], 4, cont_keep, [])    

[[4, 1, 2, 3], [1, 4, 2, 3], [1, 2, 4, 3], [1, 2, 3, 4]]

In [ ]:
def perm(xs, cont, state):
  if not xs:
    return cont(xs, state)
  else:
    x, *xs = xs
    return perm(xs, lambda ys, st2: ins(ys, x, cont, st2), state)

perm([1,2,3,4], cont_keep, [])   

### Searching Tree Paths

Given a binary tree, find all paths from the root to a leaf.

         10
        /  \
       /    \
      5     15
     / \   /  
    1   7 12


In [ ]:
t =  [10, 
      [5,  [ 1, None, None], [ 7, None, None]], 
      [15, [12, None, None], None]
     ]

In [ ]:
def paths(node, cont):
  if node is None:
    cont([])
  else:
    value, left, right = node
    cont1 = lambda path: cont([value]+path)
    if left is None and right is None:
      paths(None, cont1)
    else:
      if left : paths(left,  cont1)
      if right: paths(right, cont1)

paths(t, cont_show)    

[10, 5, 1]; [10, 5, 7]; [10, 15, 12]; 

## Backtracking via Combinators


[Allison90](http://www.allisons.org/ll/Publications/1990BCJ/) uses the term _generator_ as a function that transforms continuations. The paper uses this concept to simulate non-deterministic computations.

Herein, a _continuation_ is a function from a (partial) state to a list of (partial) states, and a _generator_ is a function from a continuation to a continuation,

    cont :: state -> [state]

    gen :: cont -> cont

<!-- original ML code for what follows (run at https://sosml.org/editor)
fun ret a = [a];

fun fail h a = [];

fun run g = g ret [];

(******************)

fun literal c h a = h( c::a );
print (run (literal 3));

fun pipe g1 g2 h a = g1(g2 h)a;
print (run (pipe (literal 1) (literal 2)));

fun success h a = h a;

fun do_ n g = if n=0 
                then success
                else pipe (do_ (n-1) g) g;
print (run (do_ 5 (literal 1) ));

fun either g1 g2 h a = (g1 h a) @ (g2 h a);

print (run (either (pipe (literal 1) (literal 2))
                   (literal 3) ) );

fun choice n = if n=0 
                 then fail
                 else either (literal n) (choice (n-1));

print ( run( do_ 3 (choice 2) ));

fun filter p h a = if p a 
                     then h a
                     else [];

fun first_one (h::t) = h=1;

print (run (pipe (do_ 3 (choice 2)) 
                 (filter first_one)));

--> 

Let's consider the generator `literal` that prepends a value to a state:

In [ ]:
literal = lambda value: lambda cont: lambda state: cont([value]+state)

To observe states, let's add

+ `ret` a continuation that just returns the given state as a list (as defined by the type)

+ `run` a function that executes a generator and returns the list of all list of states

In [ ]:
ret = lambda x: [x]

run = lambda gen: gen(ret)([])

Let's try them with `literal`,

In [ ]:
run(literal(1))

[[1]]

Generator `pipe` is used to create a generator that is a sequence of two generators,

In [ ]:
def pipe(gen1, gen2):
  return lambda cont: lambda state: gen1(gen2(cont))(state)

In [ ]:
g1 = literal(1)
g2 = literal(2)
g3 = pipe(g1, g2) # g1 only after g2, like function composition

run(g3)

[[2, 1]]

We can also implement a n-ary pipe (while gladly taking the chance of using `reduce`):

In [ ]:
from functools import reduce

def pipes(*gens):
  def f(cont):
    def g(state, gens=gens):
      gen, *gens = gens
    # new_cont = gen2(gen3(gen4(...genN(cont))))
      new_cont = reduce(lambda acc, g: g(acc), gens[::-1], cont)
      return gen(new_cont)(state)
    return g
  return f

In [ ]:
g4 = pipes(g1, g2, literal(4), g3)
run(g4)

[[2, 1, 4, 2, 1]]

The next two generators are useful for backtracking purposes in defining a successful branch and a failed branch,

In [ ]:
success = lambda cont: lambda state: cont(state)
fail    = lambda cont: lambda state: []

Generator `do` pipes a given generator a given number of times,

In [ ]:
def do(gen, n):
  if n==0:
    return success
  return pipe(do(gen, n-1), gen)

In [ ]:
g5 = do(g3, 3)  
run(g5)

[[2, 1, 2, 1, 2, 1]]

Generator `either` is the one introducing non-determinism into the computation. It receives a list of generators and produces a generator which behaves as a non-determinist choice between all given generators.

The continuation is passed to all generators, in depth-first search style.

In [ ]:
from operator import add

# def either(gen1, gen2):
#   """ just two generators """
#   return lambda cont: lambda state: gen1(cont)(state) + gen2(cont)(state)

def either(*gens):
  return lambda cont: lambda state: add(*[gen(cont)(state) for gen in gens])

One use case of `either`: generator `choice` prepends values $n$ to $1$ to all given states,

In [ ]:
def choice(n):
  if n==0:
    return fail
  return either(literal(n), choice(n-1))

To produce the cartesian product of values $1,2,3$

In [ ]:
g6 = do(choice(3), 3)
run(g6)

[[3, 3, 3],
 [2, 3, 3],
 [1, 3, 3],
 [3, 2, 3],
 [2, 2, 3],
 [1, 2, 3],
 [3, 1, 3],
 [2, 1, 3],
 [1, 1, 3],
 [3, 3, 2],
 [2, 3, 2],
 [1, 3, 2],
 [3, 2, 2],
 [2, 2, 2],
 [1, 2, 2],
 [3, 1, 2],
 [2, 1, 2],
 [1, 1, 2],
 [3, 3, 1],
 [2, 3, 1],
 [1, 3, 1],
 [3, 2, 1],
 [2, 2, 1],
 [1, 2, 1],
 [3, 1, 1],
 [2, 1, 1],
 [1, 1, 1]]

The next generator filters states by predicate:

In [ ]:
def filt(p):
  return lambda cont: lambda state: cont(state) if p(state) else []

The next use case filters the previous cartesian product by selecting only those states starting with value $1$,

In [ ]:
first_one_only = lambda st: st[0]==1

run(pipe(g6, filt(first_one_only)))

[[1, 3, 3],
 [1, 2, 3],
 [1, 1, 3],
 [1, 3, 2],
 [1, 2, 2],
 [1, 1, 2],
 [1, 3, 1],
 [1, 2, 1],
 [1, 1, 1]]

Let's produce the Pythagorean triples up to a certain limit:

In [ ]:
def less_than(state):
  return len(state)<2 or state[0]<state[1]

def is_pyth(state):
  a,b,c = state
  return a*a + b*b == c*c

def pyth_triples(limit):
  pick_number = pipe(choice(limit), filt(less_than))
  pick_3 = do(pick_number, 3)
  return run(pipe(pick_3, filt(is_pyth)))

pyth_triples(50)  

[[14, 48, 50],
 [30, 40, 50],
 [27, 36, 45],
 [9, 40, 41],
 [24, 32, 40],
 [15, 36, 39],
 [12, 35, 37],
 [21, 28, 35],
 [16, 30, 34],
 [18, 24, 30],
 [20, 21, 29],
 [10, 24, 26],
 [7, 24, 25],
 [15, 20, 25],
 [12, 16, 20],
 [8, 15, 17],
 [9, 12, 15],
 [5, 12, 13],
 [6, 8, 10],
 [3, 4, 5]]

### The n-Queen problem

Let's check how to use these combinators to solve the famous n-queen problem.

The next standard Python function will be used to prune invalid branches,

In [ ]:
def valid(state):
  """ checks if first queen does not threat the other queens """
  for i in range(1,len(state)):
    if ( state[0] == state[i] or       # if same line
         abs(state[0]-state[i]) == i): # or same diagonal
      return False
  return True

With all these tools, backtracking the n-queen problem can be solved by applying the following combinators:

In [ ]:
def queens(n):
  place_a_queen = pipe(choice(n), filt(valid))
  return run(do(place_a_queen, n))

print(*queens(6), sep='\n')

[2, 4, 6, 1, 3, 5]
[3, 6, 2, 5, 1, 4]
[4, 1, 5, 2, 6, 3]
[5, 3, 1, 6, 4, 2]


## Making Streams

[Allison90](http://www.allisons.org/ll/Publications/1990BCJ/) also presents a way to implement streams as continuations. A stream is a function which produces the first value of a sequence and another stream; the latter represents the rest of the sequence.

Herein, there are three concepts:

+ Sources produce the first value, and is given a sink to use that value

+ Agents receive value and send new modified values

+ Sinks consume a value, and is given a source that is used to produce more values.

Their types are:

     source :: sink -> list
     sink :: int -> source -> list
     agent :: sink -> sink

The next implementations are very recursive intensive. In Python it's easy to max the call stack. These functions are more appropriate for languages with automatic tail-call optimization.

Some basic sources and sinks:

In [ ]:
null_src = lambda sink: []

# basic sinks
collect  = lambda n: lambda src: [n]+src(collect)
dev_null = lambda n: lambda src: src(dev_null) # abandon hope all ye who enter here

The first source produces values from $a$ to $b$, a bit like Python's `range`.

In [ ]:
# interval :: (int, int) -> source
def interval(a, b):
  def f(sink):
    if a <= b:
      return sink(a)(interval(a+1, b))
    else:
      return []
  return f

In [ ]:
print(interval(1,10)(collect))
print(interval(1,10)(dev_null))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[]


Function `run` links a source, an agent and a link:

In [ ]:
# run :: source -> agent -> source
# run :: source -> agent -> sink -> list
run = lambda src: lambda agent: lambda sink: src(agent(sink))

The next functions implement agents that return even and odd indexed stream values (streams here are 1-indexed),

In [ ]:
def even(sink):
  return lambda n: lambda src: src(odd(sink))

def odd(sink):
  return lambda n: lambda src: sink(n)(lambda sink2: src(even(sink2)))

In [ ]:
run(interval(1,10))(odd)(collect)

[1, 3, 5, 7, 9]

Agent `pipe` creates a sequence of two agents,

In [ ]:
# pipe :: agent -> agent -> agent
def pipe(agent1, agent2):
  return lambda sink: lambda n: lambda src: agent1(agent2(sink))(n)(src)

In [ ]:
run(interval(1,10))(pipe(odd, even))(collect)

[3, 7]

`alternate` takes two sources and creates a new one that alternates between their values,

In [ ]:
# alternate :: (source, source) -> source
def alternate(src1, src2):
  return lambda sink: src1(lambda n: lambda src1a: 
                                sink(n)(alternate(src2, src1a)))

In [ ]:
s1 = interval(11,20)
s2 = interval(21,30)
print(alternate(s1,s2)(collect))

[11, 21, 12, 22, 13, 23, 14, 24, 15, 25, 16, 26, 17, 27, 18, 28, 19, 29, 20, 30]


Funciton `tee` makes two copies of a given source,

In [ ]:
# tee :: source -> (source, source)
def tee(src):
  return (src, src)

In [ ]:
print(alternate(*tee(s1))(collect))

[11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20]


Function `split` is an inverse function of `alternate`,

In [ ]:
# split :: source -> (source, source)
def split(src):
  return run(src)(odd), run(src)(even)

In [ ]:
s1 = interval(11,20)
s2, s3 = split(s1)
print(alternate(s3,s2)(collect))

[12, 11, 14, 13, 16, 15, 18, 17, 20, 19]


`filt` filters the received values by a given predicate,

In [ ]:
# filt :: (int -> bool) -> agent
def filt(p):
  def f(sink):
    def g(n):
      def h(src):
        if p(n):
          return sink(n)(run(src)(filt(p)))
        else:
          return run(src)(filt(p))(sink)
      return h
    return g
  return f

In [ ]:
run(interval(1,10))(filt(lambda n: n%2==0))(collect)

[2, 4, 6, 8, 10]

As a typical example in stream-land, let's source the ancient Sieve of Eratosthenes,

In [ ]:
# sieve :: agent 
def sieve(sink):
  return lambda n: lambda src: sink(n)(run(src)(pipe(filt(lambda m: m%n!=0), sieve)))

In [ ]:
run(interval(2,60))(sieve)(collect)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]

Some ending words from the article:

> Continuations allow the programmer to specify the flow of control of a program (this is why they are useful in denotational semantics). They allow backtracking programs such as the generators to be written. They also allow the stream processing functions - sources, sinks and agents - to pass control between each other as coroutines or as processes. At first sight functional languages seem to be poor in control mechanisms, possessing only if, application, composition and recursion. In reality this simplicity is richness and continuations are one way of getting a desired control regime.
>
> Generators and stream processing are important models in solving certain types of problem. Continuations bring them into pure functional programming without the need for new language mechanisms.
>
> Continuations involve a novel style of programming but the programs that result (n-queens, primes) are often close to their English descriptions. They have something of an imperative flavour - do this and then do that - but they are functional and free of side-effects. The programs can be efficiently implemented by optimisations such as tail-recursion. 

## A Logic Checker

One application where continuations are useful is search.

Denys Duchier shows [here](https://www.ps.uni-saarland.de/~duchier/python/continuations.html) that search can be achieved by passing _two_ continuations:

+ a success continuation, used to progress the search

+ a failure continuation, to backtrack to a previous choice

Something like

```python
  def search(state, yes, no):
    if check(state):
      return yes(state, no)
    else:
      return no()
```

To implement searching by a second path if the first failed,

```python
  def search(state, yes, no):
    if check(state):
      return path1.search(state, 
                          yes, 
                          lambda state,yes,no: path2.search(info, yes, no))
```


The next program, written by Denys Duchier, shows how to use this method to find logical assignments to validate a boolean expression.

In [ ]:
# Copyright (c) Feb 2000, by Denys Duchier, Universitaet des Saarlandes

"""
This module implements a validity checker for propositional
formulae.  Its purpose is to illustrate programming with
continuations to implement a `backtracking' search engine.

A formula is represented by an object which responds to
the following methods:

    self.satisfy(environment,yes,no)
    self.falsify(environment,yes,no)

`environment' is a partial assignment of truth values to propositional
variables. `satisfy' attempts to make the formula true, possibly
by appropriately extending the partial assignment. `no' is the
failure continuation.  It takes no argument, and resumes search
in an alternative branch of an earlier choice point. `yes' is the
success continuation and takes 2 arguments: the current partial
assignment environment, and the current failure continuation.
"""

The superclass is `Formula` that includes method `isValid` that tries to falsify an expression. If it can't, then the expression is a tautology.

In [15]:
class Formula:
  def isValid(self):
    """a formula is valid iff it cannot be falsified"""
    return self.falsify({},
                        lambda environment,no: False,
                        lambda               : True)
    
  _tracing = id # print

  # satisfy and falsify are wrappers that allow tracing
  # _satisfy and _falsify do the actual work
  def satisfy(self, environment, yes, no):
    Formula._tracing(f'satisfy {self} environment:{environment}')
    return self._satisfy(environment, yes, no)

  def falsify(self, environment, yes, no):
    Formula._tracing(f'falsify {self} environment:{environment}')
    return self._falsify(environment, yes, no)

The next subclasses implement how to satisfy or falsify the respective logical operator:

In [3]:
class Negation(Formula):
  def __init__(self, p):
    self.p = p

  def __str__(self):
    return f'¬{self.p}'

  def _satisfy(self, environment, yes, no):
    """to satisfy ¬P we must falsify P"""
    return self.p.falsify(environment, yes, no)

  def _falsify(self, environment, yes, no):
    """to falsify ¬P we must satisfy P"""
    return self.p.satisfy(environment, yes, no)

In [5]:
class Conjunction(Formula):
  def __init__(self, p, q):
    self.p = p
    self.q = q

  def __str__(self):
    return f'({self.p} ∧ {self.q})'

  def _satisfy(self, environment, yes, no):
    """to satisfy P∧Q we must satisfy both P and Q"""
    return self.p.satisfy(
      environment,
      lambda environment,no,self=self,yes=yes: 
        self.q.satisfy(environment, yes, no),
      no
    )

  def _falsify(self, environment, yes, no):
    """to falsify P∧Q we can falsify either P or Q"""
    return self.p.falsify(
      environment, 
      yes,
      lambda self=self,environment=environment,yes=yes,no=no: 
        self.q.falsify(environment, yes, no)
    )

In [6]:
class Disjunction(Formula):
  def __init__(self,p,q):
    self.p = p
    self.q = q

  def __str__(self):
    return f'({self.p} ∨ {self.q})'

  def _satisfy(self, environment, yes, no):
    """to satisfy P∨Q we can satisfy either P or Q"""
    return self.p.satisfy(
      environment, 
      yes,
      lambda self=self,environment=environment,yes=yes,no=no: 
        self.q.satisfy(environment, yes, no)
    )

  def _falsify(self, environment, yes, no):
    """to falsify P∨Q we must falsify both P and Q"""
    return self.p.falsify(
      environment,
      lambda environment,no,self=self,yes=yes: 
        self.q.falsify(environment, yes, no),
      no
    )

Class `Variable` is where the environment is populated with assignments

In [7]:
class Variable(Formula):
  def __init__(self,v):
    self.v = v

  def __str__(self):
    return self.v

  def bind(self, value, environment):
    """returns a new partial assignment that additionally
       assigns the truth 'value' to this propositional variable"""
    environment = environment.copy()
    environment[self.v] = value
    return environment

  def assign(self, value, environment, yes, no):
    """attempts to assign the given truth value to this proposition
       variable.  If environment already contains a contradictory
       assignment, the failure continuation is invoked. Otherwise, environment
       is extended if necessary and the success continuation is invoked."""
    if self.v in environment:
      return yes(environment, no) if environment[self.v]==value else no()
    else:
      return yes(self.bind(value, environment), no)

  def _satisfy(self, environment, yes, no):
    """to satisfy a propositional variable, we must assign it true"""
    return self.assign(True, environment, yes, no)

  def _falsify(self, environment, yes, no):
    """to falsify a propositional variable, we must assign it false"""
    return self.assign(False, environment, yes, no)

The next auxiliary functions are useful to facilitate the writing of logical formulas:

In [8]:
def AND(*args):
  """n-ary version of Conjunction"""
  formula, *args = args
  for x in args:
    formula = Conjunction(formula,x)
  return formula

def OR(*args):
  """n-ary version of Disjunction"""
  formula, *args = args
  for x in args:
    formula = Disjunction(formula,x)
  return formula

def NOT(x):
  return Negation(x)

def IF(p,q):
  return OR(NOT(p),q)

def IFF(p,q):
  return AND(IF(p,q), IF(q,p))

def XOR(p,q):
  return OR(AND(p,NOT(q)), AND(NOT(p),q))

A use case:

In [16]:
P = Variable('P')
Q = Variable('Q')
R = Variable('R')

# is [(P∨Q) ∧ (P⇒R) ∧ (Q⇒R)] ⇒ R a tautology?
form = IF(AND(OR(P,Q), 
              IF(P,R), 
              IF(Q,R)), 
          R)

Formula._tracing = print # hack to see what's happening under the hood
form.isValid()

falsify (¬(((P ∨ Q) ∧ (¬P ∨ R)) ∧ (¬Q ∨ R)) ∨ R) environment:{}
falsify ¬(((P ∨ Q) ∧ (¬P ∨ R)) ∧ (¬Q ∨ R)) environment:{}
satisfy (((P ∨ Q) ∧ (¬P ∨ R)) ∧ (¬Q ∨ R)) environment:{}
satisfy ((P ∨ Q) ∧ (¬P ∨ R)) environment:{}
satisfy (P ∨ Q) environment:{}
satisfy P environment:{}
satisfy (¬P ∨ R) environment:{'P': True}
satisfy ¬P environment:{'P': True}
falsify P environment:{'P': True}
satisfy R environment:{'P': True}
satisfy (¬Q ∨ R) environment:{'P': True, 'R': True}
satisfy ¬Q environment:{'P': True, 'R': True}
falsify Q environment:{'P': True, 'R': True}
falsify R environment:{'P': True, 'R': True, 'Q': False}
satisfy R environment:{'P': True, 'R': True}
falsify R environment:{'P': True, 'R': True}
satisfy Q environment:{}
satisfy (¬P ∨ R) environment:{'Q': True}
satisfy ¬P environment:{'Q': True}
falsify P environment:{'Q': True}
satisfy (¬Q ∨ R) environment:{'Q': True, 'P': False}
satisfy ¬Q environment:{'Q': True, 'P': False}
falsify Q environment:{'Q': True, 'P': False}
satisf

True

# References

+ Nick Mose - [Understanding Recursion and Continuation with Python](https://coderscat.com/understanding-recursion-and-continuation-with-python/)

+ Eli Bendersky - [On Recursion, Continuations and Trampolines](https://eli.thegreenplace.net/2017/on-recursion-continuations-and-trampolines/)

+ Denys Duchier - [Continuations Made Simple and Illustrated](https://www.ps.uni-saarland.de/~duchier/python/continuations.html)

+ L. Allison - [Continuations, Implement Generators, and Streams](http://www.allisons.org/ll/Publications/1990BCJ/)

+ Sven-Olof Nyström - [Continuations](https://www.it.uu.se/edu/course/homepage/avfunpro/ht10/notes/html/f06-cont.html)

+ Tiago Cogumbreiro - [CS 450: Structure of Higher Level Languages](https://cogumbreiro.github.io//teaching/cs450/s20/), lecture 32

<!--

## Implementing a Prolog [draft]

Another implementation by Denys Duchier. Didn't have time to understand it, but I'm placing it here, for now, since the source code is already not available at the original link.

# Copyright (c) Feb 2000, by Denys Duchier, Universitaet des Saarlandes

"""
This modules implements a rudimentary prolog engine.  Its
purpose is to illustrate the use of continuations to program
a search engine with backtracking and cut.
"""

def bind(var,term,environment):
  """bind var to term in environment environment. return the updated
  environment.  we make a copy so that we don't have to undo on
  backtracking (in essence: we always trail)."""
  environment = environment.copy()
  environment[var]=term
  return environment

def unify(t1,t2,environment,yes,no):
  """attempt to unify t1 with t2 in environment environment.
  yes is the success continuation. no is the failure continuation"""
  t1 = t1.deref(environment)
  t2 = t2.deref(environment)
  if t1 is t2:
    return yes(environment,no)
  elif t1.isVar():
    return yes(bind(t1,t2,environment),no)
  elif t2.isVar():
    return yes(bind(t2,t1,environment),no)
  elif t1.fun!=t2.fun or len(t1.args)!=len(t2.args):
    return no()
  else:
    return unifyN(len(t1.args)-1,t1.args,t2.args,environment,yes,no)

def unifyN(index,list1,list2,environment,yes,no):
  """attempt to unify to sequences of equal lengths"""
  if index<0:
    return yes(environment,no)
  else:
    return unify(list1[index],list2[index],environment,
                 lambda				\
                 environment,no,			\
                 list1=list1,list2=list2,	\
                 index=index-1,yes=yes:		\
                 unifyN(index,list1,list2,environment,yes,no),
                 no)

class Term:
  def deref(self,environment):
    return self
  def isVar(self):
    return 0
  def collectVars(self,list):
    pass
  def instantiate(self,environment,topvars,allvars):
    return self

class Var(Term):
  def __init__(self,name):
    self.name = name
  def deref(self,environment):
    if self in environment:
      return environment[self].deref(environment)
    else:
      return self
  def isVar(self):
    return 1
  def __str__(self):
    return '?'+str(self.name)
  def collectVars(self,list):
    if not (self in list):
      list.append(self)
  def rename(self,environment):
    if self in environment:
      return environment[self]
    else:
      v2 = Var(self.name)
      environment[self] = v2
      return v2
  def solve(self,engine,environment,yes,no,entryno):
    t = self.deref(environment)
    if t is self:
      raise "cannot call an uninstantiated literal"
    else:
      return t.solve(engine,environment,yes,no,entryno)
  def instantiate(self,environment,topvars,allvars):
    t = self.deref(environment)
    if t is self:
      if self in topvars:
        return self
      else:
        i = 1
        while 1:
          name = '%s:%d' % (self.name,i)
          if name not in allvars:
            allvars[name]=1
            var = Var(name)
            environment[self]=var
            topvars.append(var)
            return var
          else:
            i = i+1
    else:
      return t.instantiate(environment,topvars,allvars)

class Cons(Term):
  def __init__(self,fun,args):
    self.fun  = fun
    self.args = args
  def __str__(self):
    l = []
    for x in self.args:
      l.append(str(x))
    return str(self.fun)+'('+','.join(l)+')'
  def collectVars(self,list):
    for x in self.args:
      x.collectVars(list)
  def new(self,fun,args):
    return Cons(fun,args)
  def rename(self,environment):
    fun2 = self.fun.rename(environment)
    args2 = []
    for x in self.args:
      args2.append(x.rename(environment))
    return self.new(fun2,args2)
  def solve(self,engine,environment,yes,no,entryno):
    return engine.call(self.fun,self,environment,yes,no,entryno)
  def instantiate(self,environment,topvars,allvars):
    l = []
    for x in self.args:
      l.append(x.instantiate(environment,topvars,allvars))
    return Cons(self.fun,l)

class Atom(Term):
  def __init__(self,name):
    self.fun  = name
    self.args = []
  def __str__(self):
    return str(self.fun)
  def __call__(self,*args):
    return Cons(self,args)
  def rename(self,environment):
    return self
  def solve(self,engine,environment,yes,no,entryno):
    return engine.call(self,self,environment,yes,no,entryno)

class Rule:
  def __init__(self,head,body):
    self.head = head
    self.body = body
  def rename(self,environment=None):
    if environment is None:
      environment = {}
    return Rule(self.head.rename(environment),
                self.body.rename(environment))

AND = Atom('AND')
OR  = Atom('OR')
CUT = Atom('CUT')

class Engine:
  """implements a prolog engine"""
  def __init__(self):
    self.db    = {}
    self.environment = None
    self.no    = None
    self.vars  = None
    self.query = None
  def rule(self,head,*body):
    body = AND(*body)
    r = Rule(head,body)
    p = head.fun
    if p in self.db:
      l = self.db[p]
    else:
      l = []
      self.db[p] = l
    l.append(r)
  def run(self,Q):
    """run a query Q"""
    self.query = Q
    self.environment = None
    self.no    = None
    self.vars  = []
    Q.collectVars(self.vars)
    yes = lambda environment,no,self=self: self.succeed(environment,no)
    no  = lambda self=self : self.fail()
    Q.solve(self,{},yes,no,no)
  def succeed(self,environment,no):
    """this is called when a solution is found"""
    self.environment = environment
    self.no    = no
    print('yes')
    # all the disgusting stuff below is just so that we can
    # print a coherent and informative answer to the query
    topvars = self.vars[:]
    allvars = {}
    for x in topvars:
      allvars[x.name]=1
    t1 = Cons(None,topvars)
    t2 = Cons(None,(self.query,))
    t3 = Cons(None,(t1,t2))
    t3 = t3.instantiate(environment,topvars,allvars)
    t1,t2 = t3.args
    print(str(t2.args[0]))
    for i in range(len(self.vars)):
      print('\t'+str(self.vars[i])+' = '+str(t1.args[i]))
  def fail(self):
    """this is called when no (more) solution is found"""
    self.environment = None
    self.no    = None
    print('no')
  def next(self):
    """invoke this to search for the next solution"""
    if self.no:
      self.no()
    else:
      print('No more')
  def call(self,pred,literal,environment,yes,no,entryno):
    """the main functor of the formula is pred.  We treat specially
    functors for AND, OR and CUT.  For all others, we look in the
    engine's database to find appropriate clauses."""
    if   pred is AND:
      return self.solveAll(0,literal.args,environment,yes,no,entryno)
    elif pred is OR :
      return self.solveSome(0,literal.args,environment,yes,no,entryno)
    elif pred is CUT:
      return yes(environment,entryno)
    else:
      if pred not in self.db:
        raise "unknown predicate "+str(pred)
      else:
        return self.execute(0,self.db[pred],literal,environment,yes,no,no)
  def solveAll(self,i,l,environment,yes,no,entryno):
    """solve all literals in sequence l"""
    n = len(l)
    if n==0:
      return yes(environment,no)
    elif i==n-1:
      return l[i].solve(self,environment,yes,no,entryno)
    else:
      return l[i].solve(
        self,environment,
        lambda environment,no,i=i+1,l=l,self=self,yes=yes,entryno=entryno:\
        self.solveAll(i,l,environment,yes,no,entryno),
        no,entryno)
  def solveSome(self,i,l,environment,yes,no,entryno):
    """solve one of the literals in sequence l"""
    n = len(l)
    if n==0:
      return no()
    elif i==n-1:
      return l[i].solve(self,environment,yes,no,entryno)
    else:
      return l[i].solve(
        self,environment,yes,
        lambda self=self,i=i+1,l=l,environment=environment,yes=yes,no=no,entryno=entryno: \
        self.solveSome(i,l,environment,yes,no,entryno),
        entryno)
  def execute(self,i,rules,literal,environment,yes,no,entryno):
    """try to solve literal using rules"""
    n = len(rules)
    if i==n-1:
      return self.tryrule(rules[i],literal,environment,yes,no,entryno)
    else:
      return self.tryrule(
        rules[i],literal,environment,yes,
        lambda i=i+1,rules=rules,literal=literal,environment=environment, \
        yes=yes,no=no,entryno=entryno,self=self: \
        self.execute(i,rules,literal,environment,yes,no,entryno),
        entryno)
  def tryrule(self,rule,literal,environment,yes,no,entryno):
    """try to solve literal using rule."""
    # we first rename all variables in the rule so that we can
    # safely use them without risk of clashing with variables
    # already in use.  then we try to unify the literal with
    # the rule's head and if that succeeds we proceed to solve
    # the body of the rule.
    rule = rule.rename()
    return unify(rule.head,literal,environment,
                 lambda environment,no,rule=rule,self=self,yes=yes,entryno=entryno : \
                 rule.body.solve(self,environment,yes,no,entryno),
                 no)

X	= Var('X')
Y	= Var('Y')
Z	= Var('Z')
L	= Var('L')
L1	= Var('L1')
L2	= Var('L2')
L3	= Var('L3')
MEMBER	= Atom('member')
NIL	= Atom('nil')
CONS	= Atom('cons')
APPEND	= Atom('append')
PERMUTE	= Atom('permute')
INSERT	= Atom('insert')
a	= Atom('a')
b	= Atom('b')
c	= Atom('c')
MEMBER1	= Atom('member1')

E = Engine()
E.rule(MEMBER(X,CONS(X,L)))
E.rule(MEMBER(X,CONS(Y,L)),MEMBER(X,L))
E.rule(APPEND(NIL,L,L))
E.rule(APPEND(CONS(X,L1),L2,CONS(X,L3)),APPEND(L1,L2,L3))
E.rule(PERMUTE(NIL,NIL))
E.rule(PERMUTE(CONS(X,L1),L3),AND(PERMUTE(L1,L2),INSERT(X,L2,L3)))
E.rule(INSERT(X,L,CONS(X,L)))
E.rule(INSERT(X,CONS(Y,L1),CONS(Y,L2)),INSERT(X,L1,L2))
E.rule(MEMBER1(X,CONS(X,L)),CUT)
E.rule(MEMBER1(X,CONS(Y,L)),MEMBER1(X,L))

E.run(MEMBER(X,L))

-->